In [4]:
import numpy as np
import pandas as pd
import GPy

from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.model_selection import train_test_split

In [5]:
import itertools
datafile = 'data/jass/rnd_01.csv'
col_names = [a+str(b) for (a,b) in itertools.product([farbe for farbe in 'HKSE'], [bild for bild in range(9)])]+["Geschoben", "Player", "Aktion"]
data = pd.read_csv(datafile, header=None, names=col_names)
data

,H0,H1,H2,H3,H4,H5,H6,H7,H8,K0,...,E2,E3,E4,E5,E6,E7,E8,Geschoben,Player,Aktion
0,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,1,1631,2
1,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,64310,6
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,16721,1
3,1,0,0,0,1,0,0,0,1,1,...,0,0,1,0,1,0,0,1,0,4
4,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,72620,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161689,0,1,0,0,0,1,0,0,0,0,...,0,1,0,1,1,0,0,1,55942,3
161690,0,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,0,0,0,0,6
161691,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,1,0,0,0,3
161692,0,1,0,1,0,0,0,0,1,1,...,1,0,0,0,0,0,1,0,60659,6


## Binary Classification on targets 0 and 1

In [6]:
# select subset of data
target_classes = [0,1]
zeroone = data[data['Aktion'].isin(target_classes)]
zeroone = zeroone.drop(columns='Player')
X = zeroone.loc[:,'H0':'Geschoben'].values
y = zeroone.loc[:,'Aktion']

# make sure targets are binary (required by GPC implementation of sklearn)
y = y.isin(target_classes[0:1]) # first class = 1, second = 0
y = y.values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=500, test_size=10000, random_state=23)
print('Train: %i, Test: %i' % (len(y_train), len(y_test)))

Train: 500, Test: 10000


### With sklearn (uses Laplace approx)

In [5]:
%%time
kernel = 1.0*RBF()
clf = GaussianProcessClassifier(kernel, n_restarts_optimizer=0, random_state=23)
_ = clf.fit(X_train, y_train)

CPU times: user 2.08 s, sys: 400 ms, total: 2.48 s
Wall time: 1.26 s


In [11]:
score = clf.score(X_test, y_test)
print('Accuracy: %.3f' % score)
print('Kernel Hyperparams:\n  variance    %10.1f\n  length_scale%10.1f\nLog Marginal Likelihood: %.1f' % (clf.kernel_.k1.get_params()['constant_value'], clf.kernel_.k2.get_params()['length_scale'], clf.log_marginal_likelihood()))

Accuracy: 0.955
Kernel Hyperparams:
  variance        6141.4
  length_scale      20.6
Log-evidence    -107.0


### With GPy (uses EP approx)

In [12]:
%%time
m = GPy.models.GPClassification(X_train,y_train.reshape(-1,1))

CPU times: user 8.33 s, sys: 3.12 s, total: 11.4 s
Wall time: 5.74 s


In [13]:
%%time
for i in range(3):
    m.optimize('bfgs', max_iters=100)

 /usr/local/lib/python3.6/dist-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


CPU times: user 38.7 s, sys: 12.9 s, total: 51.5 s
Wall time: 25.8 s


In [18]:
pred_prob_means = m.predict(X_test)[0].reshape(-1)
pred = pred_prob_means > 0.5
score = np.equal(pred, y_test).mean()
print('Accuracy: %.3f' % score)
print('Kernel Hyperparams:\n  variance    %10.1f\n  length_scale%10.1f\nLog Marginal Likelihood: %.1f: %.1f' % (m.kern.variance, m.kern.lengthscale, m.log_likelihood()))

Accuracy: 0.955
Kernel Hyperparams:
  variance        1873.4
  length_scale      31.8
Log-evidence: 109.2


### Increasing training set size

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=1000, test_size=10000, random_state=23)
print('Train: %i, Test: %i' % (len(y_train), len(y_test)))

Train: 1000, Test: 10000


In [20]:
%%time
m = GPy.models.GPClassification(X_train,y_train.reshape(-1,1))

CPU times: user 1min 18s, sys: 27.1 s, total: 1min 45s
Wall time: 52.9 s


In [21]:
%%time
for i in range(3):
    m.optimize('bfgs', max_iters=100)

 /usr/local/lib/python3.6/dist-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


CPU times: user 6min 48s, sys: 2min 21s, total: 9min 9s
Wall time: 4min 45s


In [22]:
pred_prob_means = m.predict(X_test)[0].reshape(-1)
pred = pred_prob_means > 0.5
score = np.equal(pred, y_test).mean()
print('Accuracy: %.3f' % score)
print('Kernel Hyperparams:\n  variance    %10.1f\n  length_scale%10.1f\nLog Marginal Likelihood: %.1f' % (m.kern.variance, m.kern.lengthscale, m.log_likelihood()))

Accuracy: 0.961
Kernel Hyperparams:
  variance        2514.7
  length_scale      33.7
Log-evidence: 169.2


## Sparse Gaussian Process

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=1000, test_size=10000, random_state=23)
print('Train: %i, Test: %i' % (len(y_train), len(y_test)))

Train: 1000, Test: 10000


In [36]:
%%time
m = GPy.models.SparseGPClassification(X_train,y_train.reshape(-1,1), num_inducing=150)

CPU times: user 9.08 s, sys: 1.97 s, total: 11 s
Wall time: 5.55 s


In [37]:
%%time
for i in range(6):
    m.optimize('bfgs', max_iters=100)
    print(m)


Name : SparseGPClassification
Objective : 374.5421538248663
Number of Parameters : 5552
Number of Optimization Parameters : 5552
Updates : True
Parameters:
  SparseGPClassification.  |              value  |  constraints  |  priors
  inducing_inputs          |          (150, 37)  |               |        
  rbf.variance             |  95.31267044176253  |      +ve      |        
  rbf.lengthscale          |   29.5454140981156  |      +ve      |        

Name : SparseGPClassification
Objective : 195.21958590615122
Number of Parameters : 5552
Number of Optimization Parameters : 5552
Updates : True
Parameters:
  SparseGPClassification.  |               value  |  constraints  |  priors
  inducing_inputs          |           (150, 37)  |               |        
  rbf.variance             |   968.1786266458541  |      +ve      |        
  rbf.lengthscale          |  36.714500459230536  |      +ve      |        

Name : SparseGPClassification
Objective : 172.42955345097926
Number of Parameter

In [45]:
pred_prob_means = m.predict(X_test)[0].reshape(-1)
pred = pred_prob_means > 0.5
score = np.equal(pred, y_test).mean()
print('Accuracy: %.3f' % score)
print('Kernel Hyperparams:\n  variance    %10.1f\n  length_scale%10.1f\nLog Marginal Likelihood: %.1f' % (m.kern.variance, m.kern.lengthscale, m.log_likelihood()))

Accuracy: 0.961
Kernel Hyperparams:
  variance         968.9
  length_scale      22.0
Log Marginal Likelihood: -169.7


The log-marginal-likelihood of the full and the sparse model is equal (~169). This implies that a sparse model with 150 inducing variables explains the data as well as the full model with 1000 training points!

## Sparse Gaussian Process (on 10'0000)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=10000, test_size=10000, random_state=23)
print('Train: %i, Test: %i' % (len(y_train), len(y_test)))

Train: 10000, Test: 10000


In [9]:
%%time
m = GPy.models.SparseGPClassification(X_train,y_train.reshape(-1,1), num_inducing=150)

CPU times: user 21min 11s, sys: 3min 54s, total: 25min 5s
Wall time: 12min 37s


In [ ]:
%%time
for i in range(6):
    m.optimize('bfgs', max_iters=100)
    print(m)


Name : SparseGPClassification
Objective : 2908.428568920057
Number of Parameters : 5552
Number of Optimization Parameters : 5552
Updates : True
Parameters:
  SparseGPClassification.  |              value  |  constraints  |  priors
  inducing_inputs          |          (150, 37)  |               |        
  rbf.variance             |  405.5641109277961  |      +ve      |        
  rbf.lengthscale          |  57.99457502433479  |      +ve      |        

Name : SparseGPClassification
Objective : 1060.853373041733
Number of Parameters : 5552
Number of Optimization Parameters : 5552
Updates : True
Parameters:
  SparseGPClassification.  |               value  |  constraints  |  priors
  inducing_inputs          |           (150, 37)  |               |        
  rbf.variance             |  407.47683576165747  |      +ve      |        
  rbf.lengthscale          |  21.795378616346625  |      +ve      |        

Name : SparseGPClassification
Objective : 1043.96645648712
Number of Parameters :

In [ ]:
pred_prob_means = m.predict(X_test)[0].reshape(-1)
pred = pred_prob_means > 0.5
score = np.equal(pred, y_test).mean()
print('Accuracy: %.3f' % score)
print('Kernel Hyperparams:\n  variance    %10.1f\n  length_scale%10.1f\nLog Marginal Likelihood: %.1f' % (m.kern.variance, m.kern.lengthscale, m.log_likelihood()))